# Introduction {#sec-introduction}

Artificial intelligence (AI) has become a headline topic in today's news, capturing the attention of individuals across various fields eager to leverage its capabilities to enhance their work. Historically, computers were introduced to take on repetitive and mundane tasks, leading to significant advancements in processing speed, communication infrastructures, and storage capacities. However, with the advent of sophisticated AI algorithms, we are now faced with a new level of challenges and solutions.

In the past, problem-solving involved identifying the issue and explicitly programming the computer to transform input data into desired outputs. Software specialists had to meticulously define every step of the problem-solving process to achieve the correct outcome. This traditional method required a deep understanding of the problem and the logic needed to solve it.

The emergence of AI, however, has revolutionized this approach. Instead of explicitly programming each step, we now provide the system with vast amounts of data, allowing it to "learn" and adjust itself to achieve the desired results. This learning process enables AI to tackle problems that were previously unsolvable using classical methods. By training models with large datasets, AI systems develop the capability to make predictions, recognize patterns, and generate insights without human intervention in the learning process.

This shift from explicit programming to AI opens up possibilities for addressing complex problems across diverse domains. The challenge now lies in selecting the appropriate type of learning—supervised, unsupervised, or reinforcement learning—based on the nature of the problem and the available data. Understanding the strengths and limitations of each learning type is essential for effectively harnessing AI to solve real-world problems. From art and entertainment to industry, manufacturing, and healthcare, AI's transformative impact is evident. These developments highlight the need for individuals and organizations to embrace AI, as leveraging its potential can drive innovation, efficiency, and improvements across various domains. As AI continues to evolve, integrating it into diverse fields becomes increasingly crucial. Therefore, understanding and utilizing AI is essential to remain competitive and progressive.

## Problem Statement

This thesis stems from a research project that explored using AI solutions to improve structural analysis, specifically by optimizing design parameters in FEM simulations. The goal is to enhance the efficiency and accuracy of engineering analyses. The thesis will further detail how AI - or more specifically machine learning - can be integrated into these workflows, aiming to identify effective models, reduce computational costs, and improve decision-making in engineering design. Individuals involved in this process are highly skilled experts, and their involvement represents a significant investment for the industry. Given the complexity and precision required in these tasks, even small improvements in efficiency can lead to substantial cost savings, increased profitability, and significant time savings. Streamlining their workflow, whether through automation or enhanced tools, has the potential not only to reduce expenses but also to accelerate project timelines, allowing companies to bring products to market faster and gain a competitive edge.

### ML\@Karoprod Project

The BMBF project titled "Machine Learning for the Prediction of Process Parameters and Component Quality in Automotive Body Production (ML\@Karoprod)" focuses on optimizing process parameters in a product chain for deforming a metal plate through a series of operations. Traditionally, an expert conducts process simulations, adjusts parameters, and evaluates the results to achieve the desired shape. This iterative process, which takes about 20 minutes per cycle, is repeated until the desired accuracy is reached, with the expert manually making the necessary adjustments and running further simulations.

![Process in action - IWU Fraunhofer Dresden](img/Chp1/karo_process.png){#fig-karo-proc fig-align="center" width="50%"}

The process begins with deep drawing, a sheet metal forming technique where a metal blank is radially drawn into a forming die by a punch, with the goal of producing a wrinkle-free and crack-free product. Our simulated deep drawing data consists of a number of valid experiments, each characterized by various process parameters such as blank holder force, insertion position, material ID, and punch ID. In collaboration with our partners, we have identified the relevant set of features that are necessary and sufficient to fully determine the effects of a simulation.

numerical the sheet thickness (from 0.99 to 1.48 mm) the blank-holder force (from 10 to 500 kN) the insertion position (-5 to +5 mm)

categorical the drawing depth (30, 50, or 70 mm) the drawing gap (1.6 or 2.4 mm)

material id stemple id (?)

![process](img/Chp1/karoprod.png){#fig-karo fig-align="center" width="50%"}

![higher quality](img/Chp1/pikaso_enhance__none_2K_Standard_r_c_.jpeg){#fig-karo-hq fig-align="center" width="50%"}

![Process Parameters for 1000 simulations](img/Chp1/table_params.png){#table-params fig-align="center" width="50%"}

![Zt](img/Chp1/z70-50-30.png){fig-align="center" width="50%"}

| Zt  | number of  Simulations | number of faces in mesh |
|-----|------------------------|-------------------------|
| 30  | 500                    | 26759                   |
| 50  | 250                    | 28587                   |
| 70  | 250                    | 31976                   |

Approximately 1000 simulation files in mesh format were generated by the project partner by varying process parameters. After data cleaning, about 880 of these files were usable. The meshes had three different geometries (with three different depths: 30, 50, and 70), and parameters such as force and material varied, resulting in different final mesh outcomes. The ultimate goal in this section was to train a neural network to predict the dependency between changes in input parameters and the generated mesh. With sufficient training, the model would be able to accurately predict the shape of the final mesh even with new inputs that were not seen during training. This allows the specialist to quickly and accurately observe the final simulation results by manipulating parameters, in a much shorter time compared to traditional finite element methods.

|                                                                                                                                 |                                                                                             |
|------------------------------------|------------------------------------|
| ![The reference mesh for extracting face centers (Drawing depth Zt=70).](img/Chp1/banana_1.png){fig-align="center" width="50%"} | ![The same mesh in a closer view](img/Chp1/banana_zoom.png){fig-align="center" width="50%"} |

## Finite Element Method

The Finite Element Method (FEM) is a robust computational technique widely used in engineering and physical sciences to solve complex problems related to structures, fluids, and thermal dynamics. The essence of FEM lies in its ability to break down a large, complicated problem into smaller, more manageable parts known as finite elements. By systematically solving these elements, FEM provides an approximate solution that represents the behavior of the entire system.

### Mathematical Foundations

FEM is essentially a numerical approximation method that provides a solution for the continuous field $u$ of any partial differential equation (PDE) defined on a specific domain $Omega$. The governing PDE can be expressed generally as:

$$
\mathcal{L}(u) = 0 \quad \text{on } \Omega
$$

with boundary conditions:

$$
u = u_d \quad \text{on } \Gamma_D
$$

$$
\frac{\partial u}{\partial x} = g \quad \text{on } \Gamma_N
$$

In these equations, $L(u)$ represents a differential operator applied to the field $u$. This operator encapsulates the specific physical laws governing the problem, such as heat conduction, elasticity, fluid flow, etc. The nature of $L(u)$ depends on the type of PDE being solved:

For thermal problems: $L$ could represent the Laplace operator $\nabla^2 u$ , describing how heat diffuses through a material.

For structural problems: $L(u)$ might include terms representing the balance of forces, such as in the elasticity equations where $L(u)$ could involve the divergence of stress tensors.

For fluid dynamics: $L(u)$ could include the Navier-Stokes equations, which describe the motion of fluid substances.

In general, $L(u) = 0$ defines a PDE that the field $u$ must satisfy over the domain $Omega$.

### Discretization and System of Equations

To solve this PDE using FEM, the domain $Omega$ is discretized into $m$ finite elements with $n$ nodes. This discretization, along with the boundary conditions, leads to a system of linear or non-linear equations. For a linear operator $L$, this system can be represented as:

$$
\begin{pmatrix}
k_{1,1} & k_{1,2} & \cdots & k_{1,n} \\
k_{2,1} & k_{2,2} & \cdots & k_{2,n} \\
\vdots & \vdots & \ddots & \vdots \\
k_{n,1} & k_{n,2} & \cdots & k_{n,n}
\end{pmatrix}
\begin{pmatrix}
u_1 \\
u_2 \\
\vdots \\
u_n
\end{pmatrix} =
\begin{pmatrix}
F_1 \\
F_2 \\
\vdots \\
F_n
\end{pmatrix}
$$

In this equation, ${K}(u)$ is the non-linear left hand side matrix, also called the stiffness matrix. ${u}$ represents the unknowns (i.e., the solution at each node), and ${F}$ is the load vector or the right-hand side of the equation.

### Solving the System: Iterative Techniques

The solution $u^h$ is obtained by solving this system of equations iteratively. A common approach is to use the Newton–Raphson method, which involves linearizing the residual of the system:

$$
r(u^h) = K(u^h)u^h - F
$$

The iterations continue until the norm of the residual $r(u^h)$ meets the desired tolerance level. For linear problems, the system converges in one iteration, but for non-linear systems, multiple iterations are generally required. The most computationally intensive step in FEM is often the solution of this linear system, particularly when dealing with a large number of elements and nodes. Given that the mesh and its structure play a critical role in the success or failure of FEM, we will address this topic in detail in the next section.

### Mesh structure and Its Impact on FEM Analysis

A mesh is a specialized type of graph, consisting of vertices (points in 3D space) and edges (connections between these vertices), which collectively form the faces that define the surface of a 3D object. For any given geometry, it is possible to generate an infinite number of meshes, making them highly versatile for representing 3D shapes. Meshes are particularly effective and popular due to their ability to accurately portray complex geometries. By increasing the mesh resolution, the quality and precision of the 3D representation are enhanced, allowing for more detailed and realistic visualizations. Dense meshes, in particular, excel at accurately representing intricate details such as corners and curves, making them essential for rendering complex 3D objects. However, higher resolution meshes come with significant trade-offs, as they require more computational power and advanced graphics cards to manage the increased data load.

![three different meshes to represent one 3D object](img/Chp1/mesh-geom.png){#fig-mesh-geom width="40%" fig-align="center"}

Another aspect of the mesh is its topology. Mesh topology refers to the arrangement and connectivity of elements, such as triangles, quadrilaterals, or tetrahedra, within a mesh. It defines how these elements are connected through their vertices and edges, focusing on the structure and relationships between elements rather than their specific geometric positions. Topology is concerned with the mesh's internal structure without considering the actual spatial coordinates of the points.

On the other hand, geometry deals with the actual spatial placement and shape of the elements in the mesh. Geometry provides the coordinates and dimensions of each element, defining the shape and size of the mesh within the physical domain. While topology describes the connectivity, geometry specifies where the vertices are positioned and how the mesh represents the physical space.

![mesh topology vs. geometry](img/Chp1/mesh-geo-topo.png){width="40%" fig-align="center"}

The relationship between topology and geometry is crucial in mesh generation. A mesh with the same topology can have different geometries based on the positioning of the vertices. Good topology is essential for accurately representing the geometry of the model, especially in complex regions. Poor topology, such as elements with bad aspect ratios or improper connectivity, can lead to inaccuracies in representing the geometry, negatively affecting the quality and convergence of the finite element analysis.

A mesh in the context of FEM is defined as a network of interconnected elements that subdivides a complex geometry into smaller, simpler parts, known as elements or cells. These elements can take various shapes, such as triangles or quadrilaterals in 2D, and tetrahedra or hexahedra in 3D. In this context, the vertices of the mesh are referred to as nodes, and the faces are known as elements. The connections between nodes, which form the edges, are typically called edges or connections in FEM. The mesh serves as the framework over which the mathematical equations governing the physical behavior of the system are solved. The quality and resolution of the mesh are crucial, as they directly influence the accuracy, convergence, and computational efficiency of FEM analyses. A finer mesh improves accuracy but requires more computational resources, while a coarser mesh reduces computational demand but lowers precision. Understanding the structure and characteristics of the mesh is essential for optimizing FEM simulations and achieving reliable results in complex engineering problems.

![a bad mesh (left) vs a good mesh (right) for FEM](img/Chp1/fem-mesh-opt.png){width="40%" fig-align="center"}

In general, suitable meshes for FEM should exhibit several essential characteristics: they must be sufficiently refined to accurately capture the geometry and stress gradients of the structure, with a higher mesh density in regions of high stress or complex geometry. The mesh elements should maintain an appropriate aspect ratio, avoiding excessively elongated or distorted shapes, to ensure numerical accuracy and stability. Additionally, the mesh should be well-aligned with the boundaries and features of the model to minimize interpolation errors and enhance the precision of the analysis. It is always important to remember that the significance of the mesh is so crucial that even with a poorly constructed mesh, we can obtain results that are vastly different and full of errors.

FEM is widely used in various industries to simulate physical phenomena and optimize designs, significantly reducing production costs by allowing for virtual testing and refinement before physical prototypes are created. The method gained prominence in the 1960s and has since undergone significant advancements, driven by improvements in computer technology and numerical algorithms. Today, FEM is integral to fields such as aerospace, automotive, civil engineering, and biomedical engineering, offering highly accurate predictions and contributing to innovation and efficiency in product development. In this thesis however, our focus on FEM is specifically related to structural analysis and deformation problems. Since this thesis aims to explore the potential of using AI methods for solving FEM simulations, a brief introduction to neural networks and their functions is necessary.

## Introduction to Deep Neural Networks

Deep Neural Networks (DeepNN) represent a significant evolution in the field of artificial intelligence, particularly in machine learning and pattern recognition. Building on the foundation of traditional neural networks, DeepNNs consist of multiple layers of interconnected neurons, allowing them to model complex patterns and relationships in data more effectively than shallow networks. The advent of DeepNNs has revolutionized numerous fields, including computer vision, natural language processing, and reinforcement learning, marking a pivotal shift in the capabilities of AI systems.

![A hierarchical diagram showing the relationship between Artificial Intelligence, Machine Learning, Neural Networks, and Deep Learning.](img/Chp1/AI_diagram.png){#fig-ai-diag fig-align="center" width="30%"}

### Perceptron

The concept of neural networks dates back to the mid-20th century, with the introduction of the Perceptron. Introduced by Frank Rosenblatt in 1958, Perceptron is a type of artificial neuron and one of the earliest models for supervised learning in neural networks. It simulates the way a single neuron in the human brain works by taking multiple input signals, assigning each a weight, summing them up, and then passing the result through an activation function to produce an output. This simple model can classify data into two categories by finding a linear decision boundary, making it foundational for understanding more complex neural network structures. Despite its limitations, such as being unable to solve problems that aren't linearly separable, the perceptron laid the groundwork for modern neural networks and deep learning.

![Perceptron](img/Chp1/perceptron.png){#fig-perceptron fig-align="center" width="30%"}

The development of the backpropagation algorithm in the 1980s was a major milestone in the field of neural networks, enabling the effective training of deep, multi-layered networks. Backpropagation, introduced by Rumelhart, Hinton, and Williams in 1986, is a method used to adjust the weights of the network in order to improve its performance. The backpropagation works as follows:

### Forward Pass

In a neural network, the input data is fed forward through the network to generate an output. This involves calculating the output of each neuron in each layer by applying a weighted sum of inputs followed by an activation function.

For a given neuron, the output can be represented as: 
$$
z_j = \sum_{i} w_{ji}*x_i + b_j
$$

where:

$z_j$ is the weighted input to neuron $j$, $w_{ji}$ is the weight connecting input $i$ to neuron $j$, $x_i$ is the input from the previous layer, $b_j$ is the bias term.

The output $a_j$ of neuron $j$ is then calculated using an activation function $\sigma$ :\
$$
a_j = \sigma(z_j)
$$

### Backward Pass (Backpropagation)

Once the forward pass is complete and the network produces an output, backpropagation is used to adjust the weights. The goal is to determine how much each weight in the network contributed to the error in the output. Backpropagation works by computing the gradient of the error with respect to each weight by propagating the error backward through the network.

The partial derivative of the output with respect to the weight $w_{ji}$ is computed using the chain rule of calculus. For a neuron in the output layer, the error gradient with respect to a weight is given by: 
$$
\frac{\partial E}{\partial w_{ji}} = \delta_j \cdot a_i
$$ where:

$E$ is the error associated with the output, $\delta_j$ is the error term for neuron $j$, $a_i$ is the output of the neuron in the previous layer.

The error term $\delta_j$ for each neuron in the output layer is: 
$$
\delta_j = \phi'(z_j) \cdot (a_j - y_j)
$$

where $y_j$ is the target output.

For neurons in the hidden layers, $delta_j$ is computed as: 
$$
\delta_j = \phi'(z_j) \sum_k \delta_k w_{kj}
$$ 
where the sum is over all neurons $k$ in the subsequent layer that receive input from neuron $j$.

### Gradient Descent

After computing the gradients, the weights are updated to minimize the error. Gradient descent is the optimization technique used to adjust the weights. In its simplest form, the weights are updated as follows: 
$$
w_{ji}^{new} = w_{ji}^{old} - \eta \frac{\partial E}{\partial w_{ji}}
$$

where: $\eta$ is the learning rate, a small positive number that controls the step size of the update,

$\frac{\partial E}{\partial w_{ji}}$ is the gradient of the error with respect to the weight.

This process is repeated iteratively across multiple epochs, gradually adjusting the weights to reduce the overall error of the network.

The error mentioned in the previous sections is formally known as the loss function $L(\theta)$, which quantifies the difference between the predicted outputs and the actual target values. Here,$\theta$ represents the network's parameters (weights and biases).

$$
\theta_{t+1} = \theta_t - \eta \nabla_\theta L(\theta_t)
$$

where $\eta$ is the learning rate, and $\nabla_\theta L(\theta_t)$ is the gradient of the loss function with respect to the parameters. The gradient $\nabla_\theta L(\theta_t)$ indicates the direction and rate of the steepest increase in the loss. By moving in the opposite direction, gradient descent reduces the loss, thereby improving the performance of the neural network. However, the success of gradient descent in deep networks relies heavily on factors such as effective weight initialization, the choice of activation functions, and strategies to mitigate challenges like overfitting and the vanishing gradient problem. The introduction of ReLU activation functions, expressed as $Relu(x)=\max(0,x)$, was instrumental in overcoming the vanishing gradient problem by ensuring that gradients remain significant in deeper layers, thus allowing for better learning and performance.

### Activation Function

An activation function in neural networks is a mathematical function applied to the output of a neuron that introduces non-linearity into the model. These functions help the network learn complex patterns by allowing it to capture non-linear relationships between inputs and outputs. Activation functions determine whether a neuron should be "activated" or "fired" based on its input, controlling the flow of information in the network. Without activation functions, the model would behave like a simple linear regression and would not be able to solve more complex tasks.

Below are some of the most important activation functions along with their formulas:

#### Linear Activation Function
The linear (or identity) activation function simply passes the input as it is without any transformation.

$$
f(x) = x
$$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
x = np.linspace(-10, 10, 400)

plt.plot(x,x)

This is used in the output layer for regression tasks but is rarely used in hidden layers due to its lack of non-linearity.

#### Sigmoid Activation Function

The sigmoid function maps the input to a range between 0 and 1. It’s widely used in binary classification problems.

$$
f(x) = \frac{1}{1 + e^{-x}}
$$

- **Range**: \( (0, 1) \)
- **Common use case**: Output layer in binary classification.


#### Hyperbolic Tangent (Tanh) Activation Function
The tanh function maps the input to a range between -1 and 1, offering a zero-centered output.

$$
f(x) = \tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}
$$

- **Range**: (-1, 1)
- **Common use case**: Hidden layers in many neural networks.


#### ReLU (Rectified Linear Unit) Activation Function
ReLU is one of the most commonly used activation functions in modern deep learning networks. It outputs the input directly if positive, otherwise, it outputs zero.

$$
f(x) = \max(0, x)
$$

- **Range**: [0, $\infty$) 
- **Common use case**: Hidden layers in convolutional and fully connected networks.


#### Leaky ReLU Activation Function
Leaky ReLU is a variation of ReLU that allows a small, non-zero gradient when the input is negative, helping mitigate the "dying ReLU" problem.

$$
f(x) = \begin{cases} x & \text{if } x > 0 \\ \alpha x & \text{if } x \leq 0 \end{cases}
$$

Where $\alpha$ is a small constant (typically 0.01).

- **Range**: (-$\infty$, $\infty$)
- **Common use case**: Hidden layers to avoid dead neurons.

### 6. **Sine Activation Function**
The sine activation function is less commonly used but can be beneficial in certain special cases or scientific models.

$$
f(x) = \sin(x)
$$

- **Range**: (-1, 1)
- **Common use case**: Experimental or specific tasks requiring periodic behavior.


These activation functions play critical roles in determining the performance of neural networks, with each function having its strengths and weaknesses depending on the task at hand.

### Loss Function

Loss functions serve as a measure of how well or poorly a model's predictions align with the actual target values. By quantifying the error or "loss" between the predicted outputs and the true labels, the loss function guides the adjustment of the model's weights during training. Essentially, the model seeks to minimize the loss function, and by doing so, it iteratively updates the weights to improve accuracy and performance.

In many cases, standard loss functions like Mean Squared Error (MSE) for regression tasks or Cross-Entropy Loss for classification tasks are sufficient. However, there are situations where a custom loss function is necessary to address specific challenges or nuances of the problem at hand. For instance, in imbalanced datasets, where certain classes are underrepresented, a custom loss function might be designed to assign higher penalties to misclassifications of the minority class, thereby ensuring the model learns more effectively. Crafting a custom loss function often requires deep domain knowledge and experience, as it needs to balance the specific objectives of the task with the overall training process. This expertise helps in fine-tuning the loss function to achieve optimal performance for the given problem.

#### Mean Absolute Error (MAE) Loss

L1 loss, also known as Mean Absolute Error (MAE), is a loss function commonly used in NN models, especially for regression tasks. It measures the average of the absolute differences between predicted values and actual values. The L1 loss is robust to outliers since it penalizes errors linearly rather than quadratically like L2 loss. Minimizing L1 loss encourages sparsity in the model's parameters, making it useful in contexts such as feature selection and sparse models. The formula for L1 loss is given as:

$$
\text{MAE} = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y_i}|
$$

Where $y$ represents the actual values, $hat{y}$ the predicted values, and $n$ is the total number of data points. The absolute value function ensures that the magnitude of the error is considered regardless of its direction.

#### Mean Squared Error (MSE) Loss

MSE loss or L2 loss, is one of the most commonly used loss functions in regression problems. It measures the average squared difference between the predicted values and the actual target values. MSE is particularly sensitive to large errors due to the squaring of the differences, meaning that models producing large errors are penalized more heavily. $$
\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

#### Combined Loss

Integrating different loss functions into deep neural networks allows for balancing various aspects of model performance. A single loss function might not capture all the essential characteristics for optimal learning. For example, one loss function $Loss_A$ might focus on robustness to outliers, while another loss function $Loss_B$ could emphasize minimizing large errors or encouraging smooth predictions. By combining these loss functions, we can leverage the strengths of both. The combined loss can be written as:

$$
\text{Combined Loss} = \alpha \cdot {Loss_A} + \beta \cdot {Loss_B}
$$

Here, $\alpha$ and $\beta$ are non-negative hyperparameters that determine how much each loss contributes to the final combined loss. This approach allows fine-tuning to achieve a desirable trade-off between different properties, such as robustness and precision, depending on the specific task.

## Hyperparameter Tuning and Optimization

### DeepNN Architectures

Neural networks are remarkably flexible, allowing various architectures and units to be connected and combined to create more complex models tailored to specific problems. For instance, by increasing or decreasing the number of layers and the number of neurons in each layer, the network's performance can be improved. Another crucial factor is the choice of activation functions, which play a significant role in the learning process. Additionally, specialized layers such as convolutional layers and recurrent layers can be added or removed depending on the data type, the complexity of the problem, and the specific requirements. These adjustments are made to achieve the most suitable architecture for the task at hand.

For instance, a Convolutional Neural Network (CNN) can be enhanced with conditional layers, forming a Conditional Convolutional Neural Network (CCNN) that generates different outputs based on additional context. This modularity adds significant power to the neural network, but it also introduces a considerable level of complexity. The goal is to design models that are as simple as possible while still providing adequate performance to solve the targeted problems effectively. A number of widely adopted neural network architectures will be covered in the following sections.

## Multi-Layer Perceptron (MLP)

MLP is a type of feedforward artificial neural network that consists of multiple layers of neurons, where each layer is fully connected to the next one. The basic unit of an MLP is the perceptron, which only computes a weighted sum of its input features and passes the result through an activation function. By adding multiple layers to a perceptron, an MLP becomes capable of solving more complex problems. MLPs typically consist of an input layer, one or more hidden layers, and an output layer. The input layer receives the initial data, the hidden layers process the data through multiple transformations, and the output layer produces the final prediction or classification. MLPs are capable of approximating complex functions and are commonly used in tasks such as classification, regression, and pattern recognition. For an MLP with a single hidden layer:

$$
z^{(1)} = W^{(1)} {x} + b^{(1)}
$$

$$
a^{(1)} = \sigma(z^{(1)})
$$

$$
z^{(2)} = W^{(2)} a^{(1)} + b^{(2)}
$$

$$
\hat{y} = \sigma(z^{(2)})
$$

Where:

$x$ is the input vector.

$W^{(1)}$ and $W^{(2)}$ are the weight matrices for the first and second layers, respectively.

$b^{(1)}$ and $b^{(2)}$ are the bias vectors for the first and second layers, respectively.

$\sigma$ is the activation function (e.g., sigmoid, ReLU, etc.).

and $\hat{y}$ is the final output (prediction) of the MLP.

## Autoencoders

An autoencoder is a type of neural network designed to learn a compressed, efficient representation of input data. It consists of two main components: an encoder and a decoder. The encoder compresses the input data into a lower-dimensional representation, often referred to as the latent space or bottleneck. The decoder then attempts to reconstruct the original input from this compressed representation. The goal is to minimize the difference between the input and the reconstructed output.

Autoencoders, used for tasks like dimensionality reduction or feature learning, can also be improved by incorporating elements such as convolutional layers, recurrent units, or Generative Adversarial Networks (GANs). This integration allows autoencoders to learn more complex data representations, enhancing their effectiveness in applications like anomaly detection or data compression

Mathematically, the process can be represented as: $$
z = f_\theta(x)
$$

$$
\hat{x} = g_\phi({z})
$$

Here, $z$ is the latent representation of the input $x$, and $\hat{x}$ is the reconstructed output. The objective of the autoencoder is to minimize the reconstruction loss, commonly measured as the mean squared error (MSE):

$$
L({x}, \hat{{x}}) = \| {x} - \hat{{x}} \|^2_2
$$

Autoencoders are particularly useful in areas such as dimensionality reduction and reconstruction, where they compress high-dimensional data into a lower-dimensional latent space and then accurately reconstruct the original data. They also excel in denoising by removing noise from corrupted data, and in anomaly detection by identifying outliers based on reconstruction errors. Additionally, autoencoders are valuable in generative modeling, contributing to the creation of new, similar data samples, which is especially beneficial in image synthesis and data augmentation.

## Convolutional Neural Networks

Convolutional Neural Networks (CNNs) are a class of deep neural networks specifically designed to process data with a grid-like structure, such as images. Unlike fully connected networks, where each neuron is connected to every neuron in the previous layer, CNNs take advantage of the spatial structure of the data by using convolutional layers. These layers apply convolutional filters to local regions of the input, which can be mathematically represented as:

$$
y = f * x + b
$$

where $y$ is the output feature map, $f$ is the filter, $*$ denotes the convolution operation, $x$ is the input, and $b$ is the bias term. This operation allows the network to detect patterns such as edges, textures, and shapes in the input data.

![caption](img/Chp1/conv.png){#fig-conv fig-align="center" width="30%"}

A typical CNN architecture consists of multiple layers, including convolutional layers, pooling layers, and fully connected layers. Convolutional layers apply filters to the input to extract features, while pooling layers perform down-sampling, reducing the spatial dimensions of the data. A common pooling operation is max pooling, defined as:

$$
y_{i,j} = \max_{m,n} (x_{i+m,j+n})
$$

where $y_{i,j}$ represents the output after pooling, and $x_{i+m,j+n}$ is the region of the input over which the pooling operation is applied. The final fully connected layers combine the features extracted by the convolutional and pooling layers to make predictions, typically using a softmax function for classification:

$$
\text{softmax}(z_i) = \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}}
$$

where $z_i$ is the input to the softmax function, and $K$ is the number of classes.

![caption](img/Chp1/CNN.png){#fig-cnn-layers fig-align="center" width="50%"}

CNNs are widely used in various applications, including image classification, object detection, and image segmentation. Their ability to automatically learn and extract features from raw pixel data has led to significant advancements in computer vision. The modularity of CNNs allows them to be easily adapted to different tasks by adjusting the architecture, such as changing the number of layers or the size of filters. This flexibility, combined with their high accuracy and efficiency, makes CNNs a cornerstone of modern deep learning applications.

## Conditional Neural Networks

Conditional Neural Networks are a specialized type of neural network where the output is conditioned not only on the input data but also on an additional context or condition. This conditioning can be represented as an additional input that influences the network's behavior, enabling it to generate different outputs depending on the given condition.

Consider a standard neural network where the input data is $x$ and the output is $y$. The network typically learns a function $f_\theta(x) = y$, where $\theta$ represents the parameters of the network. In a Conditional Neural Network, an additional condition $c$ is introduced, modifying the function to:

$$
f_{\theta}({x}, {c}) = {y}
$$

Here, $c$ is the condition that influences the network's output. This condition could be a class label, a set of parameters, or any other contextual information relevant to the task.

If a neural network is designed to perform a specific task on a dataset, and we want to generalize that task to different categories of data, we can introduce a condition or context to the network. This allows the network to adapt its behavior based on the category or condition, making it more versatile. Moreover, this conditioning is not tied to a specific architecture; for example, a condition can be added to a convolutional network or any other architecture. For instance, in a conditional image generation task, $x$ might represent a latent vector, and $c$ could be a label corresponding to the class of the image to be generated. The network would then generate an image $y$ conditioned on both $x$ and $c$:

$$
{y} = G_{\theta}({x}, {c})
$$

Where $G_\theta$ is the generator function of the Conditional Neural Network.

The loss function in a Conditional Neural Network often takes the condition into account as well. For example, in a supervised learning scenario, the loss function $L$ could be defined as:

$$
L_\theta = \frac{1}{N} \sum_{i=1}^{N} \ell\left(f_{\theta}({x}_i, {c}_i), {y}_i\right)
$$

Where $\ell$ is a suitable loss function (e.g., cross-entropy or mean squared error), $N$ is the number of training examples, and $y_i$ is the true output corresponding to input $x_i$ under condition $c_i$.

Conditional Neural Networks are widely used in various applications, such as conditional image generation, style transfer, and structured prediction tasks. Their ability to incorporate context or conditions makes them highly versatile and effective for problems where the output must be tailored based on specific criteria. This flexibility allows conditions to be added to various architectures, whether it's a convolutional network or any other type, broadening their applicability.

## Research Questions

-   replacing mesh processing with implicit representations to have a continuous representation of models

-   continuous fields

-   infinite resolution

-   we need to deal with 3D data and NNs , so we need to find the best representation to keep the net size as small as possible even for more complicated geometries, good accuracy and most importantly be able to work with dynamics of a 3d shape

-   reduce the time of simulations

-   provide accurate set of process parameters